In [1]:
import psycopg2 as ps
import pandas as pd

conn = ps.connect(host="localhost", port = 5432, database="orioks", user="postgres", password="123456")

In [2]:
def make_request(query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()


In [6]:
def make_request_and_fetch(query):
    cursor = conn.cursor()
    cursor.execute(query)
    analysis_results = cursor.fetchall()
    for row in analysis_results:
        print(row[0])
    cursor.close()

## Задание 1

In [3]:

query = 'create table test(id serial primary key, code_1 varchar(64), code_2 varchar(64));'
make_request(query)

In [4]:
plpgsql_script = """
DO
$$
BEGIN
FOR i IN 1..1000000 LOOP
INSERT INTO Test(code_1,code_2) VALUES(md5(random()::text),
md5(random()::text));
END LOOP;
END
$$ language plpgsql;
"""

make_request(plpgsql_script)

In [14]:

query = 'explain analyze INSERT INTO test(code_1,code_2) VALUES(md5(random()::text), md5(random()::text));'
make_request_and_fetch(query)


Insert on test  (cost=0.00..0.04 rows=0 width=0) (actual time=0.044..0.045 rows=0 loops=1)
  ->  Result  (cost=0.00..0.04 rows=1 width=296) (actual time=0.027..0.028 rows=1 loops=1)
Planning Time: 0.021 ms
Execution Time: 0.058 ms


In [15]:

query = 'explain analyze select * from test order by code_1;'
make_request_and_fetch(query)


Gather Merge  (cost=73496.01..170725.10 rows=833334 width=70) (actual time=1443.844..2650.939 rows=1000004 loops=1)
  Workers Planned: 2
  Workers Launched: 2
  ->  Sort  (cost=72495.98..73537.65 rows=416667 width=70) (actual time=1389.550..1844.729 rows=333335 loops=3)
        Sort Key: code_1
        Sort Method: external merge  Disk: 26520kB
        Worker 0:  Sort Method: external merge  Disk: 25368kB
        Worker 1:  Sort Method: external merge  Disk: 26504kB
        ->  Parallel Seq Scan on test  (cost=0.00..16512.67 rows=416667 width=70) (actual time=0.007..15.997 rows=333335 loops=3)
Planning Time: 0.657 ms
Execution Time: 2673.737 ms


In [16]:

query = 'create index code1_index on test (code_1);'
make_request(query)

In [17]:


query = 'explain analyze INSERT INTO test(code_1,code_2) VALUES(md5(random()::text), md5(random()::text));'
make_request_and_fetch(query)

Insert on test  (cost=0.00..0.04 rows=0 width=0) (actual time=1.191..1.191 rows=0 loops=1)
  ->  Result  (cost=0.00..0.04 rows=1 width=296) (actual time=0.021..0.021 rows=1 loops=1)
Planning Time: 0.020 ms
Execution Time: 1.204 ms


In [18]:

query = 'explain analyze select * from test order by code_1;'
make_request_and_fetch(query)

Index Scan using code1_index on test  (cost=0.42..93223.41 rows=1000004 width=70) (actual time=0.051..393.695 rows=1000005 loops=1)
Planning Time: 0.752 ms
Execution Time: 410.305 ms


In [12]:

query = 'drop index code1_index;'
make_request(query)

| До индексации              | После индексации          |
|----------------------------|---------------------------|
| **Вставка значения**       | **Вставка значения**      |
| Planning Time: 0.021 ms    | Planning Time: 0.020 ms   |
| Execution Time: 0.058 ms   | Execution Time: 1.204 ms  |
| **Вывод значений**         | **Вывод значений**        |
| Planning Time: 0.657 ms    | Planning Time: 0.752 ms   |
| Execution Time: 2673.737 ms| Execution Time: 410.305 ms|


## Задание 2

In [19]:

query = 'explain analyze select * from test where code_1 like \'a%\' order by code_1;'
make_request_and_fetch(query)

Gather Merge  (cost=20400.82..26293.36 rows=50504 width=70) (actual time=203.433..265.505 rows=62865 loops=1)
  Workers Planned: 2
  Workers Launched: 2
  ->  Sort  (cost=19400.79..19463.92 rows=25252 width=70) (actual time=120.567..123.332 rows=20955 loops=3)
        Sort Key: code_1
        Sort Method: external merge  Disk: 2744kB
        Worker 0:  Sort Method: quicksort  Memory: 1703kB
        Worker 1:  Sort Method: quicksort  Memory: 1683kB
        ->  Parallel Seq Scan on test  (cost=0.00..17554.35 rows=25252 width=70) (actual time=0.018..28.752 rows=20955 loops=3)
              Filter: ((code_1)::text ~~ 'a%'::text)
              Rows Removed by Filter: 312380
Planning Time: 1.347 ms
Execution Time: 267.912 ms


## Задание 3

In [ ]:

query = 'create index students_id_index on student (student_id);'
make_request(query)

## Задание 4

Аномалии, которые могут возникнуть при использовании уровня изоляции READ COMMITTED, включают:

*Аномалия Non-Repeatable Read (Неповторяющееся чтение)*: Студент дважды считывает оценку и видит разные значения. Например, если преподаватель выставил оценку между двумя запросами студента, студент в запросах получит две разные оценки.

*Фантомное чтение*: Студент делает выборку своих оценок, а в этот момент преподаватель выставляет оценку. Таким образом, если в рамках первой транзакции снова будет выполнен запрос
на выборку, то результат будет отличаться от первого.

*Аномалия сериализации*. В данном случае результат выполнения нескольких
транзакций последовательно (при любом порядке их следования) не будет
совпадать с параллельным их выполнением.


Транзакция 1:

Чтение оценки студента.
Выставление новой оценки.
Транзакция 2:

Чтение оценки того же студента.
Пусть Транзакция 1 сначала читает оценку студента и планирует выставить новую оценку, а Транзакция 2 читает оценку после Транзакции 1.

Порядок операций:
Транзакция 1: Чтение старой оценки.
Транзакция 2: Чтение старой оценки.
Транзакция 1: Выставление новой оценки.
В этом сценарии, Транзакция 2 прочитает старую оценку, затем Транзакция 1 выставит новую оценку. Таким образом, результат, который получит Транзакция 2, будет отличаться от последовательного выполнения этих операций.


## Задание 5